# OpenSearch 한글 형태소 분석기 통한 키워드 검색 
>이 노트북은,
> - SageMaker Studio* **`Data Science 3.0`** kernel 및 ml.m5.large 인스턴스에서 테스트 되었습니다.
> - SageMaker Notebook **`conda_python3`** 에서 테스트 되었습니다.


여기서는 OpenSearch 가 설치된 것을 가정하고, 한글 형태소 분석기의 사용하는 법을 알려 드립니다.

---

### [중요]
- 이 노트북은 Bedrock Titan Embedding Model 을 기본으로 사용합니다. 
- 오픈 서치 서비스가 액티브 된 상태를 가정 합니다.
- 앞서 02_OpenSearch_setup 노트북의 clean-up 이전 부분까지 모두 완료가 되어 있어야 합니다.


---
## Ref: 
- [Amazon OpenSearch Service로 검색 구현하기](https://catalog.us-east-1.prod.workshops.aws/workshops/de4e38cb-a0d9-4ffe-a777-bf00d498fa49/ko-KR/indexing/blog-reindex)
- [OpenSearch Python Client](https://opensearch.org/docs/1.3/clients/python-high-level/)
- [OpenSearch Match, Multi-Match, and Match Phrase Queries](https://opster.com/guides/opensearch/opensearch-search-apis/opensearch-match-multi-match-and-match-phrase-queries/)
- OpenSearch Query 에서 Filter, Must, Should, Not Mush 에 대한 설명 입니다.
    - [OpenSearch Boolean Queries](https://opster.com/guides/opensearch/opensearch-search-apis/opensearch-boolean-queries/#:~:text=Boolean%20queries%20are%20used%20to,as%20terms%2C%20match%20and%20query_string.)
- [OpenSearch Query Description (한글)](https://esbook.kimjmin.net/05-search)


## 1. 환경 세팅

In [1]:
import boto3
region = boto3.Session().region_name
opensearch = boto3.client('opensearch', region)

%store -r opensearch_user_id opensearch_user_password domain_name opensearch_domain_endpoint

try:
    opensearch_user_id
    opensearch_user_password
    domain_name
    opensearch_domain_endpoint
   
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Run 00_setup notebook first or Create Your Own OpenSearch Domain")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%store

Stored variables and their in-db values:
domain_name                            -> 'ebp-poc-all'
opensearch_domain_endpoint             -> 'https://search-ebp-poc-all-uw3oqtjpbgjeg4tbb5pf3i
opensearch_user_id                     -> 'raguser'
opensearch_user_password               -> 'Passw0rd1!'


### Bedrock Client 생성

In [4]:
import boto3
import os
import json
from botocore.config import Config
import botocore 
from pprint import pprint
from termcolor import colored

session = boto3.Session()

retry_config = Config(
    region_name=os.environ.get("AWS_DEFAULT_REGION", None),
    retries={
        "max_attempts": 10,
        "mode": "standard",
    },
)

# modelId = "anthropic.claude-instant-v1"  # (Change this to try different model versions)
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
accept = "application/json"
contentType = "application/json"

bedrock = boto3.client(service_name='bedrock')
boto3_bedrock = boto3.client(service_name='bedrock-runtime',config=retry_config)

model_list = bedrock.list_foundation_models()
result = [(fm_list["modelName"], fm_list["modelId"]) for fm_list in model_list["modelSummaries"] if fm_list['inferenceTypesSupported'] == ['ON_DEMAND']]
pprint(result)

[('Titan Text Large', 'amazon.titan-tg1-large'),
 ('Titan Text Embeddings v2', 'amazon.titan-embed-g1-text-02'),
 ('Titan Text G1 - Lite', 'amazon.titan-text-lite-v1'),
 ('Titan Text G1 - Express', 'amazon.titan-text-express-v1'),
 ('Titan Embeddings G1 - Text', 'amazon.titan-embed-text-v1'),
 ('Titan Multimodal Embeddings G1', 'amazon.titan-embed-image-v1'),
 ('Titan Image Generator G1', 'amazon.titan-image-generator-v1'),
 ('SDXL 0.8', 'stability.stable-diffusion-xl'),
 ('SDXL 0.8', 'stability.stable-diffusion-xl-v0'),
 ('SDXL 1.0', 'stability.stable-diffusion-xl-v1'),
 ('J2 Grande Instruct', 'ai21.j2-grande-instruct'),
 ('J2 Jumbo Instruct', 'ai21.j2-jumbo-instruct'),
 ('Jurassic-2 Mid', 'ai21.j2-mid'),
 ('Jurassic-2 Mid', 'ai21.j2-mid-v1'),
 ('Jurassic-2 Ultra', 'ai21.j2-ultra'),
 ('Jurassic-2 Ultra', 'ai21.j2-ultra-v1'),
 ('Claude Instant', 'anthropic.claude-instant-v1'),
 ('Claude', 'anthropic.claude-v2:1'),
 ('Claude', 'anthropic.claude-v2'),
 ('Claude 3 Sonnet', 'anthropic.clau

## 2. Titan Embedding 및 LLM 인 Claude-3 sonnet 모델 로딩

### LLM 로딩 (Claude-v3 sonnet)

In [5]:
from langchain_community.chat_models import BedrockChat
from langchain_core.messages import HumanMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [6]:
llm_text = BedrockChat(
    model_id=modelId,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs={
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "temperature" : 0,
        "top_k": 350,
        "top_p": 0.999
    }
)
llm_text

BedrockChat(client=<botocore.client.BedrockRuntime object at 0x7fa8692edc00>, region_name='us-west-2', model_id='anthropic.claude-3-sonnet-20240229-v1:0', model_kwargs={'anthropic_version': 'bedrock-2023-05-31', 'max_tokens': 4096, 'temperature': 0, 'top_k': 350, 'top_p': 0.999}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7fa88e390280>])

In [7]:
prompt1 = "나는 인공지능 AI 보험 서비스입니다. 생명과 손해 보험의 차이에 대해 설명해 주세요."
messages = [
    HumanMessage(content=prompt1)
]

# messages = [
#     {"role": "user", "content": [{"type": "text", "text": prompt1}]},
# ]

response1 = llm_text.invoke(messages)

생명보험과 손해보험은 보험의 주요 유형으로 다음과 같은 차이점이 있습니다.

1. 보장 대상
- 생명보험은 사람의 생명과 관련된 위험을 보장합니다. 예를 들어 사망, 상해, 질병 등을 대상으로 합니다.
- 손해보험은 재산상의 손해나 배상책임을 보장합니다. 예를 들어 화재, 자동차사고, 배상책임 등을 대상으로 합니다.

2. 보험금 지급 사유
- 생명보험은 피보험자의 사망, 상해, 질병 등 인적 위험이 발생했을 때 보험금을 지급합니다.
- 손해보험은 재물의 손해나 제3자에 대한 배상책임이 발생했을 때 실제 입은 손해를 보상합니다.

3. 보험기간
- 생명보험은 일반적으로 장기계약이며, 종신보험의 경우 피보험자 종신까지 보장됩니다.
- 손해보험은 단기계약이 일반적이며, 1년 만기로 갱신하는 경우가 많습니다.

4. 보험료 산정 기준
- 생명보험료는 피보험자의 나이, 건강상태, 가입금액 등에 따라 결정됩니다.
- 손해보험료는 보험목적물의 가액, 위험률, 보상한도 등에 따라 결정됩니다.

요약하면 생명보험은 개인의 생명과 관련된 위험을 장기적으로 보장하고, 손해보험은 재산상 손해나 배상책임을 단기적으로 보상하는 차이가 있습니다.

### Embedding 모델 선택

In [8]:
from langchain_community.embeddings import BedrockEmbeddings

llm_emb = BedrockEmbeddings(
    client=boto3_bedrock,
    # model_id="cohere.embed-multilingual-v3"
    model_id="amazon.titan-embed-g1-text-02"
)

-------------------

## 3. OpenSearch 벡터 Index 생성
### 선수 조건
- 랭체인 오프서처 참고 자료
    - [Langchain Opensearch](https://python.langchain.com/docs/integrations/vectorstores/opensearch)

### 오픈 서치 인덱스 유무에 따라 삭제
오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [9]:
from opensearchpy import OpenSearch, RequestsHttpConnection
http_auth = (opensearch_user_id, opensearch_user_password)
os_client = OpenSearch(
            hosts=[
                {'host': opensearch_domain_endpoint.replace("https://", ""),
                 'port': 443
                }
            ],
            http_auth=http_auth, # Master username, Master password,
            use_ssl=True,
            verify_certs=True,
            connection_class=RequestsHttpConnection
        )

### index 생성

In [10]:
index_name = "ebp-poc-all-index"
exists = os_client.indices.exists(index_name)

if exists:
    os_client.indices.delete(index=index_name)
    print("Index is deleted")
else:
    print("Index does not exist")

Index does not exist


In [11]:
## metadata, text, vector_field 의 네이밍은 langchain에서 지정된 이름
### model에 따라 dimension 사이즈 변경 필요 (Titan : 1536, Cohere : 1024)
import json

with open('index_body_simple.json', 'r') as f:
    index_body = json.load(f)

print(json.dumps(index_body, indent=2))


{
  "settings": {
    "index.knn": true,
    "index.knn.algo_param.ef_search": 512
  },
  "mappings": {
    "properties": {
      "metadata": {
        "properties": {
          "source": {
            "type": "keyword"
          },
          "type": {
            "type": "keyword"
          },
          "timestamp": {
            "type": "date"
          }
        }
      },
      "vector_field": {
        "type": "knn_vector",
        "dimension": 1536,
        "method": {
          "engine": "faiss",
          "name": "hnsw",
          "parameters": {
            "ef_construction": 512,
            "m": 16
          },
          "space_type": "l2"
        }
      }
    }
  }
}


In [12]:
os_client.indices.create(index_name, body=index_body)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'ebp-poc-all-index'}

In [13]:
%%time
from langchain_community.vectorstores import OpenSearchVectorSearch

vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
)

CPU times: user 7.44 ms, sys: 0 ns, total: 7.44 ms
Wall time: 12.2 ms


## 4. 데이터 준비


In [14]:
import time
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import PyPDFium2Loader


from langchain_core.documents import Document

# from llmsherpa.readers import LayoutPDFReader

In [15]:
import glob

data_path = './all/*'
pdf_list = glob.glob(data_path)
pdf_list

['./all/2020_국제_신재생에너지_정책변화_및_시장분석.pdf',
 './all/2022_국제_신재생에너지_정책변화_및_시장분석.pdf',
 './all/2021_국제_신재생에너지_정책변화_및_시장분석.pdf']

In [16]:
from multiprocessing.pool import ThreadPool
from multiprocessing import  Manager

import pdfplumber

In [17]:
import re

def prune_text(text, current_pdf_file):

    def replace_cid(match):
        print(f"Please check PDF file {current_pdf_file} : {match}")
        ascii_num = int(match.group(1))
        try:
            return chr(ascii_num)
        except:
            return ''  # In case of conversion failure, return empty string

    # Regular expression to find all (cid:x) patterns
    cid_pattern = re.compile(r'\(cid:(\d+)\)')
    pruned_text = re.sub(cid_pattern, replace_cid, text)
    return pruned_text

In [18]:
from datetime import datetime

def read_pdf(param):
    vector_db = param[0]
    current_pdf_file = param[1]
    print(f"current_pdf_file : {current_pdf_file}")
    docs = []
    source_name = current_pdf_file.split('/')[-1]
    type_name = source_name.split('_')[0]
    
    with pdfplumber.open(current_pdf_file) as pdf:
        for page_number, page in enumerate(pdf.pages, start=1):
            page_text = page.extract_text()
            if page_text:
                pruned_text = prune_text(page_text, current_pdf_file)
            else:
                pruned_text = ""
            if len(pruned_text) >= 20:  ## 임의로 20 이상인 sentence만 뽑도록 함
                chunk = Document(
                    page_content=pruned_text.replace('\n',' '),
                    metadata={
                        "source" : source_name,
                        "type": type_name,
                        "timestamp": datetime.now()
                    }
                )
                #print(f"chunk : {chunk}")
                docs.append(chunk)
    if len(docs) > 0 :
        vector_db.add_documents(docs)

In [19]:
manager = Manager()
result_dict = manager.dict()

# ml.m5.xlarge에서 multiprocessing으로 동작 확인
param = [(vector_db, current_pdf_file)for current_pdf_file in pdf_list]

num_processes = len(pdf_list)%os.cpu_count()

if num_processes == 0 :
    num_processes = os.cpu_count() - 1

print(f"num of process : {num_processes}")

with ThreadPool(processes=num_processes) as pool:
    pool.map(read_pdf, param)
    pool.close()
    pool.join()

num of process : 3
current_pdf_file : ./all/2020_국제_신재생에너지_정책변화_및_시장분석.pdf
current_pdf_file : ./all/2022_국제_신재생에너지_정책변화_및_시장분석.pdf
current_pdf_file : ./all/2021_국제_신재생에너지_정책변화_및_시장분석.pdf
Please check PDF file ./all/2021_국제_신재생에너지_정책변화_및_시장분석.pdf : <re.Match object; span=(43, 54), match='(cid:48823)'>
Please check PDF file ./all/2021_국제_신재생에너지_정책변화_및_시장분석.pdf : <re.Match object; span=(70, 81), match='(cid:48824)'>
Please check PDF file ./all/2021_국제_신재생에너지_정책변화_및_시장분석.pdf : <re.Match object; span=(147, 158), match='(cid:48823)'>
Please check PDF file ./all/2021_국제_신재생에너지_정책변화_및_시장분석.pdf : <re.Match object; span=(180, 191), match='(cid:48824)'>
Please check PDF file ./all/2021_국제_신재생에너지_정책변화_및_시장분석.pdf : <re.Match object; span=(224, 235), match='(cid:48823)'>
Please check PDF

### OpenSearch에 생성된 인덱스의 구성 확인

In [20]:
index_info = os_client.indices.get(index=index_name)
print(json.dumps(index_info, indent=2))

{
  "ebp-poc-all-index": {
    "aliases": {},
    "mappings": {
      "properties": {
        "metadata": {
          "properties": {
            "source": {
              "type": "keyword"
            },
            "timestamp": {
              "type": "date"
            },
            "type": {
              "type": "keyword"
            }
          }
        },
        "text": {
          "type": "text",
          "fields": {
            "keyword": {
              "type": "keyword",
              "ignore_above": 256
            }
          }
        },
        "vector_field": {
          "type": "knn_vector",
          "dimension": 1536,
          "method": {
            "engine": "faiss",
            "space_type": "l2",
            "name": "hnsw",
            "parameters": {
              "ef_construction": 512,
              "m": 16
            }
          }
        }
      }
    },
    "settings": {
      "index": {
        "replication": {
          "type": "DOCUMENT"
        },

## 5. 어휘를 기반으로 한 전문(full-text) 검색 - Lexical Search

> query의 문장은 Nori 형태소 분석기에 의해 형태소로 분리됨
> OpenSearch에 입력된 (PDF에서 추출한) 문장(청크)들과 BM25 알고리즘을 기반으로 가장 유사한(score가 높은) 값부터 상위 (k=5)개의 결과가 나타남  

In [21]:
from opensearch_dsl import Search

In [22]:
def result_to_dataframe(response):
    import pandas as pd

    pd.set_option('display.max_columns', 150)
    pd.set_option('display.max_colwidth', None)

    result = []
    for res in response['hits']['hits']:
        # print(res.keys())
        result.append([res['_index'], round(res['_score'], 4), res['_source']['metadata']['type'], res['_source']['text']])
    df = pd.DataFrame(result, columns=['index_name', 'score', 'type', 'text'])
    return df.style.set_properties(**{'text-align': 'left'})

In [23]:
def query_lexical(query, filter=[], k=5):
    QUERY_TEMPLATE = {
        "size": k,
        "query": {
            "bool": {
                "must": [
                    {
                        "match": {
                            "text": query
                        }
                    }
                ],
                "filter": filter
            }
        }
    }
    if len(filter) > 0:
        QUERY_TEMPLATE["query"]["bool"]["filter"].extend(filter)
    return QUERY_TEMPLATE

In [24]:
query = "2020년 LCOE 값을 알려줘."

response_lexical_only = os_client.search(
    body=query_lexical(query),
    index=index_name
)

time_took_lexical_only = response_lexical_only['took']
print('검색에 걸린 시간: ', time_took_lexical_only, 'ms')

print("<<사용자 입력 쿼리 문장>>: ", query)

result_to_dataframe(response_lexical_only)

검색에 걸린 시간:  86 ms
<<사용자 입력 쿼리 문장>>:  2020년 LCOE 값을 알려줘.


,index_name,score,type,text
0,ebp-poc-all-index,8.486300,2021,"2. 재생에너지원별 가격 동향 2.1. 원별 균등화발전비용(LCOE)42) 2020년은 코로나19로 인한 전 세계적인 영향을 받았으나 재생에너지 보급 비용 이 하락하는 추세에는 큰 영향을 주지 못하였다. 2020년 균등화발전비용(levelized cost of electricity, 이하 LCOE)를 보면 전년 대비 태양광은 7% 하락하였고, 해상 풍력과 육상풍력은 각각 9%, 13% 하락하였다. 집광형 태양열발전(Concentrated Solar Power, 이하 CSP)은 상대적으로 적은 보급이지만 가장 많은 16%의 가격하 락을 보였다([그림 2-27]). [그림 2-27] 유틸리티급 재생에너지 원별 LCOE 변화(2019~2020년) 주: CSP의 경우 2018~2020년 연평균 변화율임. 자료: IRENA(2021b), p.25. 재생에너지 비용은 태양광과 풍력의 경우 기술의 발전, 규모의 경제, 가치사슬의 경쟁, 개발사의 경험 축적 등으로 지난 10년간 급속히 하락하였다. 가장 비용이 많이 하락한 태양광의 경우 LCOE는 2010년 $0.381/kWh에서 2020년 $0.057/kWh로 85% 하락하였다. 육상풍력과 해상풍력 비용도 빠르게 하락하였는데 육상풍력의 경 우 동기간 $0.089/kWh에서 $0.039/kWh로 56%, 해상풍력은 $0.162 /kWh에서 $0.084/kWh로 48% 하락한 것으로 나타났다. CSP의 경우 보급이 많지 않지만 비 42) LCOE는 발전량 한 단위당 평균 발전비용으로 발전시설 운영 총비용의 현재 값을 총발전량의 현재 값으로 나누어 계산함. 26"
1,ebp-poc-all-index,6.560200,2021,"는 것이다. 동기간 육상풍력의 학습률은 32%, 해상풍력의 학습률은 15%에 이른 다.44) [그림 2-28]은 유틸리티급 재생에너지 원별 LCOE 추세를 연도별로 보여주는 것 이다. 가장 비용이 많이 하락한 태양광의 경우 추세가 안정적으로 매년 하락하였음 이 눈에 띈다. 그림에서 회색밴드는 화석연료 발전비용 범위를 나타내는 것으로 LCOE 범위가 회색밴드에 포함될 경우, 해당 재생에너지원의 발전비용이 화석연료 발전비용 범위 내에 속하게 되는 것으로 화석연료와 비교하여 경쟁력을 갖춘 것으 로 해석할 수 있다. 2020년 현재 모든 재생에너지원의 LCOE가 화석연료 발전비용 범위 내에 들어와서, 재생에너지원은 이미 화석연료 대비 가격 경쟁력을 갖춘 것으 로 볼 수 있다. 특히 육상풍력의 경우 화석연료 발전비용 최저보다 낮은 수준이다. [그림 2-28] 유릴리티급 재생에너지 원별 LCOE 변화(2010~2020년) 자료: IRENA(2021b), p.33. 44) 자료: IRENA(2021b), p.19. 28"
2,ebp-poc-all-index,5.815600,2022,"1.3. 재생에너지원별 균등화발전비용(LCOE)8) IRENA(2022)에 따르면 2021년에는 원자재와 물류비용 상승 등의 이슈가 있었 지만, 태양광, 육상풍력, 해상풍력 등의 비용 하락 추세는 유지되었다. 태양광은 부품 가격이 상승하였지만 균등화발전비용(Levelized Cost of Electricity, 이하 LCOE) 하락 추세에는 큰 영향을 미치지 못하였고, 육상풍력은 전년대비 이용율이 크게 개 선되면서 비용이 하락하였다.9) 2021년 LCOE를 보면 태양광은 전년대비 13% 감소 하였고, 해상풍력은 13%, 육상풍력은 15% 감소하였지만, CSP(Concentrating solar power)는 7% 증가하였다.10) CSP는 전세계적으로 3건의 설비가 가동 개시 되었는데, 이중 칠레의 CSP 설비가 건설 지연으로 비용이 증가하여 2010년~2015년 사이에 개발된 프로젝트와 비용이 비슷한 수준을 보였다.11) 재생에너지 비용은 지난 11년 동안 지열과 수력을 제외하고 모두 감소하면서, 재 생에너지원 모두 화석발전 비용과 비교하여 경쟁력을 갖춘 것으로 나타났다([그림 2-9]). LCOE가 가장 많이 하락한 에너지원은 태양광으로, 2010년 $0.417/kWh에 서 2021년 $0.048/kWh로 88% 하락하였다. 2021년 육상풍력 LCOE는 $0.033/kWh, 해상풍력은 $0.075/kWh로 지난 11년 동안 각각 68%와 60% 하락 한 반면, 지열과 수력의 LCOE는 각각 $0.068/kWh, $0.048/kWh로 지난 11년동 안 34%와 24% 상승하였다. 바이오는 지난 11년 동안 가격 분산이 감소하고 있으 나 상승이나 하락 등의 추세는 관측되지 않았다(<표 2-1>). 여기서 태양광과 육상풍 력의 가격하락은 기술개발의 이유가 가장 크고, 해상풍력은 최근 중국의 해상풍력 발전설비 설치 비중이 2021년 82%까지 증가함에 따라 중국의 상황이 대폭 반영된 특징이 있다.12) 2021년 원자재 가격은 상승하였지만 태양광과 육상풍력 LCOE가 하락할 수 있었 던 이유는 다음과 같다. 첫째, 원자재 가격 상승은 2020년 하반기에서 2021년 상 반기에 이루어졌으며, 2021년 가동개시된 설비는 원자재 가격 상승 이전에 건설되 었을 가능성이 높다. 특히 유럽 외 지역에서 건설된 대규모 프로젝트들은 구매력이 8) LCOE는 발전량 한 단위당 평균 발전비용으로 발전시설 운영 총비용의 현재 값을 총발전량의 현재 값으로 나누어 계산함(공지영· 조일현, 2021, p.26) 9) IRENA(2022), p.16. 10) 전게서, p.16. 11) 다만 해당 설비는 대규모 설비이므로 LCOE는 2건의 중국 CSP 설비에 비해 약간 높은 수준에 그쳤음(IRENA, 2022, p.33) 12) IRENA(2022), pp.30~31. 12"
3,ebp-poc-all-index,5.585500,2022,"<표 4-2> 변수 설명 변수명 설명 출처 전체 발전설비 용량 중 태양광, 풍력, 수력, 재생에너지 비중 Enerdata 조력, 지열, 바이오매스 발전 용량 비중 전체 발전설비 용량 중 태양광, 풍력 발전 변동성 재생에너지 비중 Enerdata 용량 비중 FiT/FiP 제도 시행시 1의 값을 갖는 FiT/FiP REN21, BNEF 더미변수 RPS RPS 제도 시행시 1의 값을 갖는 더미변수 REN21, BNEF 경매 경매제도 시행시 1의 값을 갖는 더미변수 REN21, BNEF 제조업 비중 GDP에서 제조업이 차지하는 비중 World Bank GDP per cap 1인당 GDP(2015년 US dollar) World Bank 금융발전도 GDP 대비 민간신용 비중 World Bank 석탄, 원유, 천연가스, 광물 및 임업제품의 천연자원 지대 비중 World Bank 지대가 GDP에서 차지하는 비중 BP Statistical Review of 전력 순수출량이 0보다 큰 경우 1의 값을 해외 계통연계 World Energy & 갖는 더미변수 Ember156) CO2 per cap 1인당 CO2 배출량 World Bank 국가 면적 국가 면적 World Bank 156) Our World in Data에서 BP Statistical Review of World Energy와 Ember 자료를 가공한 데이터를 활용함. 78"
4,ebp-poc-all-index,5.581000,2021,"제2장 국제 신재생에너지 보급 및 가격 동향 [그림 2-32] BNEF 추정 LCOE 변화(2010~2020년) 자료: BNEF(2021g), p.4. 2.2. 지역별 LCOE 오늘날 전 세계 인구의 2/3가 육상 풍력 또는 대규모 태양광(또는 둘 다)이 가장 저렴한 신규 대규모 전력원인 국가에 거주하고 있다. 하지만 여전히 일본, 한국, 인 도네시아, 말레이시아, 필리핀은 석탄이 가장 저렴한 신규 대규모 발전원이다([그림 2-33]). 기존 화석연료 전원의 한계 비용(기존 설비 운영)을 비교했을 때도 중국, 인 도, 독일, 프랑스, 이탈리아, 스페인, 포르투갈, 그리스의 경우 현재 기존 화석연료 발전소(석탄 및 가스)를 가동하는 것보다 신규 대규모 태양광을 건설하는 것이 더 저렴하고, 스웨덴 영국, 네덜란드

### 결과값에 매칭되는 Term 확인

BM25 알고리즘에 의해 스코어링이 되고, PDF에서 추출된 문장(chunk)에서 어떤 term들과 매칭되었는지를 확인하려면 아래의 코드를 실행합니다. 아래 쿼리는 결과값에 매칭되는 term을 강조할 수 있도록 html 태그를 추가합니다.

In [25]:
def query_lexical_with_highlight(query, filter=[], k=5):
    QUERY_TEMPLATE = {
        "size": k,
        "query": {
            "bool": {
                "must": [
                    {
                        "match": {
                            "text": query
                        }
                    }
                ],
                "filter": filter
            }
        },
        "highlight": {
            "pre_tags": [
                "<span style='color:red'>"
            ],
            "post_tags": [
                "</span>"
            ],
            "fields": {
                "text": {}
            }
        }
    }
    if len(filter) > 0:
        QUERY_TEMPLATE["query"]["bool"]["filter"].extend(filter)
    return QUERY_TEMPLATE

response_lexical_with_highlight = os_client.search(
    body=query_lexical_with_highlight(query),
    index=index_name
)


In [26]:
from IPython.display import HTML

import pandas as pd
temp_arr = []

for res in response_lexical_with_highlight['hits']['hits']:
            # result.append([res['_index'], round(res['_score'], 4), res['_source']['metadata']['type'], res['_source']['text']])

    temp_arr.append([res['_score'], res['highlight']['text']])

# print("---------- html 태그가 포함된 결과 ------------")
# print(temp_arr)
# print("-------------------------------------------")
    
#df = pd.DataFrame(temp_arr)
df = pd.DataFrame(temp_arr, columns=['score', '각 문서(chuck) 내에서 매칭된 부분'])

print("<<사용자 입력 쿼리 문장>>: ", query)

HTML(df.to_html(escape=False))


<<사용자 입력 쿼리 문장>>:  2020년 LCOE 값을 알려줘.


,score,각 문서(chuck) 내에서 매칭된 부분
0,8.486279,"[원별 균등화발전비용(LCOE)42) 2020년은 코로나19로 인한 전 세계적인 영향을 받았으나 재생에너지 보급 비용 이 하락하는 추세에는 큰 영향을 주지 못하였다. 2020년 균등화발전비용, (levelized cost of electricity, 이하 LCOE)를 보면 전년 대비 태양광은 7% 하락하였고, 해상 풍력과 육상풍력은 각각 9%, 13% 하락하였다., [그림 2-27] 유틸리티급 재생에너지 원별 LCOE 변화(2019~2020년) 주: CSP의 경우 2018~2020년 연평균 변화율임., 가장 비용이 많이 하락한 태양광의 경우 LCOE는 2010년 $0.381/kWh에서 2020년 $0.057/kWh로 85% 하락하였다., CSP의 경우 보급이 많지 않지만 비 42) LCOE는 발전량 한 단위당 평균 발전비용으로 발전시설 운영 총비용의 현재 값을 총발전량의 현재 값으로 나누어 계산함. 26]"
1,6.560203,"[동기간 육상풍력의 학습률은 32%, 해상풍력의 학습률은 15%에 이른 다.44) [그림 2-28]은 유틸리티급 재생에너지 원별 LCOE 추세를 연도별로 보여주는 것 이다., 그림에서 회색밴드는 화석연료 발전비용 범위를 나타내는 것으로 LCOE 범위가 회색밴드에 포함될 경우, 해당 재생에너지원의 발전비용이 화석연료 발전비용 범위 내에 속하게 되는 것으로, 화석연료와 비교하여 경쟁력을 갖춘 것으 로 해석할 수 있다. 2020년 현재 모든 재생에너지원의 LCOE가 화석연료 발전비용 범위 내에 들어와서, 재생에너지원은 이미 화석연료 대비, [그림 2-28] 유릴리티급 재생에너지 원별 LCOE 변화(2010~2020년) 자료: IRENA(2021b), p.33. 44) 자료: IRENA(2021b), p.19. 28]"
2,5.815553,"[재생에너지원별 균등화발전비용(LCOE)8) IRENA(2022)에 따르면 2021년에는 원자재와 물류비용 상승 등의 이슈가 있었 지만, 태양광, 육상풍력, 해상풍력 등의 비용 하락, 태양광은 부품 가격이 상승하였지만 균등화발전비용(Levelized Cost of Electricity, 이하 LCOE) 하락 추세에는 큰 영향을 미치지 못하였고, 육상풍력은 전년대비, 첫째, 원자재 가격 상승은 2020년 하반기에서 2021년 상 반기에 이루어졌으며, 2021년 가동개시된 설비는 원자재 가격 상승 이전에 건설되 었을 가능성이 높다., 특히 유럽 외 지역에서 건설된 대규모 프로젝트들은 구매력이 8) LCOE는 발전량 한 단위당 평균 발전비용으로 발전시설 운영 총비용의 현재 값을 총발전량의 현재 값으로 나누어 계산함]"
3,5.585457,"[Enerdata 조력, 지열, 바이오매스 발전 용량 비중 전체 발전설비 용량 중 태양광, 풍력 발전 변동성 재생에너지 비중 Enerdata 용량 비중 FiT/FiP 제도 시행시 1의 값을, 갖는 FiT/FiP REN21, BNEF 더미변수 RPS RPS 제도 시행시 1의 값을 갖는 더미변수 REN21, BNEF 경매 경매제도 시행시 1의 값을 갖는 더미변수 REN21, 천연가스, 광물 및 임업제품의 천연자원 지대 비중 World Bank 지대가 GDP에서 차지하는 비중 BP Statistical Review of 전력 순수출량이 0보다 큰 경우 1의 값을]"
4,5.581002,"[제2장 국제 신재생에너지 보급 및 가격 동향 [그림 2-32] BNEF 추정 LCOE 변화(2010~2020년) 자료: BNEF(2021g), p.4. 2.2., 지역별 LCOE 오늘날 전 세계 인구의 2/3가 육상 풍력 또는 대규모 태양광(또는 둘 다)이 가장 저렴한 신규 대규모 전력원인 국가에 거주하고 있다., 특히 2020년 하반기 이후 유럽 대부분 지역에서 신규 태 양광, 풍력이 기존 화석연료 한계비용보다 더 낮은 LCOE를 달성하였다.]"


참고> hightlight 요청은 모든 결과를 return하지 않고, 매칭되는 부분만 중점적으로 보여줍니다. 위의 코드에서는 검색에 매칭되는 term의 앞 뒤로 \<span style='color:red'>과 \</span>태그로 눈에 들어오도록 표기했습니다.

## 6. Filter 활용
- document내 metadata를 활용하여 search space를 줄일 수 있다.
- 특히 filter의 경우 search 전에 수행되기 때문에, 검색 속도 향상을 기대할 수 있다
- syntax
    - filter=[{"term"**[고정]**: {"metadata.source"**[메타데이터 이름, 혹은 메타데이터 아니여도 상관없음]**: "신한은행"**[조건명]**}},]
    - list 형식으로 복수개 filter 설정 가능

In [27]:
filter = [
    {"term": {"metadata.source": "2020_국제_신재생에너지_정책변화_및_시장분석.pdf"}}
]

response = os_client.search(
    body=query_lexical(query, filter),
    index=index_name
)
result_to_dataframe(response)

,index_name,score,type,text
0,ebp-poc-all-index,4.973200,2020,"다른 나라와 비교할 때 우리나라는 아직 태양광과 풍력 LCOE 비용이 높은 편이다. BNEF의 2020년 상반기 추정치를 보면, 고정식 태양광 LCOE 범위가 낮은 U.A.E는 $23~51/MWh, 인도는 $27~45/MWh이다. 육 상풍력의 경우 브라질은 $24~44MWh, 미국은 $26~59/MWh로 비용이 낮 은 것을 확인할 수 있다([그림 2-11]). 이에 반해 우리나라의 경우 고정 식 태양광 LCOE는 $73~146/MWh, 육상풍력 LCOE는 $78~142/MWh로, 가장 낮은 국가의 LCOE와 비교하여 상당히 차이가 있다([그림 2-11]). 재생에너지 비용이 높은 아시아로 좁혀서 보면 Wood Mackenzie 추정 으로 재생에너지 프리미엄이44) 평균적으로 16%였다.45) [그림 2-12]에 서 볼 수 있듯 2020년 현재 일본과 말레이시아의 재생에너지 프리미엄 이 가장 높은 국가이고, 신남방 전책으로 주목받고 있는 동남아시아는 전반적으로 프리미엄이 높다([그림 2-12]). 중국과 우리나라의 경우 프 리미엄이 존재하나 높지 않은 것으로 평가되었는데 중국의 경우 5%,46) 우리나라의 경우 4%로 추정하였다.47) 반면, 호주와 인도의 경우 앞선 BNEF의 추정과 같이 재생에너지가 가장 싼 지역으로 재생에너지 프리 미엄이 존재하지 않는다([그림 2-12]). 하지만, 2030년경이면 Wood Mackenzie 추정으로 아시아 태평양 지역 도 평균적으로 재생에너지가 화석 연료와 비교하여 23% 비용이 낮아질 것으로 추정하고 있으며 동남아시아는 경우 재생에너지 가격이 화석연 료와 비슷해질 것으로 전망하고 있다.48) 44) 태양광과 풍력 중 가장 낮은 LCOE가 석탄이나 가스 등 화석연료의 가장 낮은 L COE와 비교하여 재생에너지 비용 프리미엄을 추정. 45) Wood Mackenzie(2020), p.8. 46) 앞선 BNEF는 중국의 재생에너지 프리미엄이 없는 것으로 추정됨. LCOE는 발표 추정 기관마다 차이가 존재함. 47) 전게서, p.30, p.32. 48) 전게서, p.9. 24"
1,ebp-poc-all-index,4.467600,2020,"표 차례 <표 2-1> 2018년 최종에너지 소비 원별 비중ꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏ11 <표 2-2> 2019년 원별 발전 비중ꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏ14 <표 2-3> 유틸리티급 재생에너지 LCOE 변화 (2009년~2019년)ꞏꞏꞏꞏ19 <표 2-4> RE100 가입 기업의 국가현황ꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏ37 <표 2-5> 일본 태양광 경매 현황ꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏ42 <표 2-6> 중국 2019년과 2020년 태양광 경매 비교ꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏ43 <표 2-7> 국가별 단기적 보급 정책 변화ꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏ48 <표 3-1> 국가/지역별 해상풍력 확대 계획ꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏ72 <표 4-1> 에너지수지(Energy Balance) 계산 기준에 따른 제로에너지 정의ꞏ85 <표 4-2> 신재생에너지 공급방식에 따른 구분ꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏ85 <표 4-3> 주요국 제로에너지 빌딩 정의ꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏ88 <표 4-4> 건물 냉난방 재생에너지 정책 직접 정책, 간접 정책ꞏꞏ104 <표 4-5> HRE 시나리오별 비교ꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏꞏ108 iv"
2,ebp-poc-all-index,4.109800,2020,"한국건설기술연구원 연구보고서. 2017. 제로에너지건축물 활성화를 위한 제도개선 및 지원 방안 연구. 한국에너지공단. 2019. 2018년 신재생에너지 산업통계(2019년 공표) 한국에너지공단. 2020a. 2019년 신ꞏ재생에너지 보급통계(2020년 공표) 결과 요약. 한국에너지공단. 2020. 신재생에너지 설비의 지원 등에 관한 지침 (신재 생에너지센터 공고 제 2020-5호). <보도 자료> 산업통상자원부, 보도/해명 자료 2020.11.02. 산업통상자원부, 보도자료 “2020년 신재생에너지 보급지원사업 공고”. 2020.03.19. 산업통상자원부, “제5차 신재생에너지 기본계획(2020-2034) 발표”. 2020.12.29. <외국 문헌> BNEF. 2018. “3Q 2018 Global PV Market Outlook” BNEF. 2019a. “Who supplied the World’s Solar Market in 2018?” BNEF. 2019b. “Heat Rodadmap Europe: An Ambition for Low-Carbon Technology” BNEF. 2020a, “Power Transition Trends 2020” BNEF. 2020b. “1H 2020 LCOE Update”. BNEF. 2020c. “Global Renewable Energy Auctions and Tenders” BNEF. 2020d. “3Q 2020 Global Auction and Tender Results and 136"
3,ebp-poc-all-index,3.930400,2020,"2.2. 지역별 LCOE 전 세계적으로 태양광과 풍력 가격이 하락하면서 신규 보급되는 발 전원 중 LCOE 기준 가장 비용이 낮은 에너지원이 태양광과 풍력인 지 역이 증가하였다. 미주와 유럽 대부분 지역에서 재생에너지원 발전비용 이 화석연료 발전비용보다 낮고, 아시아에서 중국과 호주도 태양광이 가장 싼 신규 발전원이다([그림 2-10]). 하지만, 우리나라와 일본을 비 롯한 동남아시아 국가에서는 여전히 석탄이 가장 싼 발전원이다([그림 2-10]). [그림 2-10] 2019년 LCOE 기준 가장 낮은 대규모 발전 전원 주: 국가별 원별 신규설비의 LCOE 측면의 가장 낮은 발전원을 표시 함. 자료: BNEF(2020b), p.11. [그림 2-11]은 지역별 대규모 고정식 태양광과 육상풍력의 국가별 LCOE를 나타낸 것이다. 세계적으로는 태양광과 풍력 비용이 하락하여 경쟁력을 확보했지만, 지역별로는 편차가 큰 편이다. 제2장 국제 신재생에너지 동향 23"
4,ebp-poc-all-index,3.923700,2020,"재생에너지 중에서 신규보급이 가장 활발히 진행되고 있는 태양광과 육 상풍력, 해상풍력의 LCOE 비용을 세부적으로 살펴보면, 공통으로 설치비용 이 하락하고 이용률이 지속적으로 증가하고 있음을 볼 수 있다(<표 2-3>). 설치비용과 이용률은 LCOE 비용을 결정하는 중요한 요소들로써, 설치비용 이 하락하면 LCOE는 하락하고, 발전량과 직접적인 관련이 있는 이용률의 증가는 LCOE를 하락시킨다. 태양광의 설치비용은 2010년 $4,702/kW에서 2019년 $995/kW로 하락하였고, 이용률은 2010년 13.8%에서 2019년 18% 로 증가하였다(<표 2-3>). 육상풍력의 설치비용은 동기간 $1,949/kW에서 $1,473/kW로 하락하였고, 이용률은 27.1%에서 35.6%로 증가하였다(<표 2- 3>). 해상풍력의 경우 설비비용은 2000년 초중반 다소 증가하기도 하였지 만([그림 2-9]), 이후 2009년부터 꾸준히 하락하여 2010년 $4,650/kW에서 2019년 $3,800/kW로 하락하였고, 같은 기간 이용률은 36.8%에서 43.5%로 증가하였다(<표 2-3>). [그림 2-8]은 유틸리티급 재생에너지 LCOE를 보여주고, 그림에서 회색밴드는 화석연료 발전비용 범위를 나타낸다. 각 신재생에너지원별 LCOE 범위가 회색밴드에 포함될 경우, 해당 신재

In [29]:
filter = [
    {"term": {"metadata.source": "2020_국제_신재생에너지_정책변화_및_시장분석.pdf"}},
    {"term": {"metadata.type": "2020년 LCOE 값을 알려줘."}},
]

response = os_client.search(
    body=query_lexical(query, filter),
    index=index_name
)
print('이전에 필터를 넣치 않고 검색에 걸린 시간: ', time_took_lexical_only, 'ms')
print('지금 필터를 넣고 검색에 걸린 시간: ', response['took'], 'ms')

print("<<사용자 입력 쿼리 문장>>: ", query)

result_to_dataframe(response)

이전에 필터를 넣치 않고 검색에 걸린 시간:  86 ms
지금 필터를 넣고 검색에 걸린 시간:  14 ms
<<사용자 입력 쿼리 문장>>:  2020년 LCOE 값을 알려줘.


,index_name,score,type,text


## 7. 벡터 검색 (knn 검색)을 활용한 검색 - Semantic Search

- query 를 제공해서 실제로 유사한 내용이 검색이 되는지를 확인 합니다.

In [30]:
def query_semantic(vector, filter=[], k=5):
    QUERY_TEMPLATE = {
        "size": k,
        "query": {                    
            "knn": {
                "vector_field": {
                    "vector": vector,
                    "k": k 
                }
            }           
        }
    }
    return QUERY_TEMPLATE

In [31]:
print("<<사용자 입력 쿼리 문장>>: ", query)

response = os_client.search(
    body=query_semantic(llm_emb.embed_query(query)),
    index=index_name
)
result_to_dataframe(response)

<<사용자 입력 쿼리 문장>>:  2020년 LCOE 값을 알려줘.


,index_name,score,type,text
0,ebp-poc-all-index,0.004100,2021,"[그림 2-31] 유틸리티급 태양광 LCOE 비용 변화(2010~2020년) 자료: IRENA(2021b), p.83. 비용 하락이 가장 두드러지고 최근 가장 보급이 활발한 태양광의 지난 10년간의 LCOE 변화를 분해해 보면 태양광 모듈비용 하락이 46%를 차지하여 가장 중요한 요인이었다. 태양광 모듈의 가격 하락은 모듈 생산의 전 가치사슬에서 기술적 진보 로 인한 것이다. 그리고 인버터, 구조물, 나머지 BoS 비용이 18%를 차지하였고 개 발 EPC, 그 외 소프트 비용이 26%를 차지하였다. 나머지는 시장이 성숙해짐에 따 라 금융비용 하락, O&M 비용 하락, 태양광 이용률이 높은 곳의 개발이 증가함에 따라 높아진 이용률이 비용하락에 기여하였다.45) 주요 기관마다 LCOE 추정은 조금씩 다르다. BNEF의 태양광과 풍력의 2021년 상반기 기준 추정을 보면,46) 고정식 태양광은 $0.48/kWh, 추적식 태양광 $0.38/kWh, 육상풍력 $0.41/kWh, 해상풍력 $0.82/kWh이다([그림 2-32]). 이는 <표 2-2>의 IRENA 추정보다는 낮다. 여기서 눈여겨볼 것은 전통적으로 추정되던 태양광과 풍 력 등 재생에너지원에 대한 LCOE와 더불어, 태양광과 풍력이 변동성 재생에너지임 에 따라 ESS의 보급도 중요해지면서 ESS의 LCOE도 추정하여 제시한다는 것이다. 2021년 상반기 ESS의 LCOE는 $138/kWh로 최근 비용이 빠르게 하락하는 것을 볼 수 있다([그림 2-32]).47) 45) IRENA(2021b), p.83. 46) BNEF는 반기로 LCOE를 업데이트함. 47) ESS는 유틸리티급 리튬이온 배터리로 충전비용을 포함하여 일일 4시간 활용 가정. 30"
1,ebp-poc-all-index,0.004000,2021,"[그림 2-33] 2021년 LCOE 기준 최저비용 신규 대규모 발전원 자료: BNEF(2021g), p.15. [그림 2-34] 2021년 LCOE 기준 신규 태양광 풍력 VS 기존 화석연료 자료: IBNEF(2021g), p.16. 32"
2,ebp-poc-all-index,0.004000,2022,"BNEF(2022e)에서는 2022년 상반기 LCOE를 분석하였는데, 전력 부문의 비용 인플레이션으로 태양광, 육상풍력, 배터리저장장치의 LCOE가 증가하였음을 제시하 였다. IRENA(2022)에서도 2021년 하반기에서 2022년 상반기 사이에 원자재와 물 류 비용 상승으로 2022년 재생에너지 비용 상승 가능성을 언급한 바 있으나 이러한 비용 상승이 전체적인 추이에 대한 변곡점이 될 것으로 전망하지는 않았다. 한편, 공급망 위기와 가격 상승에도 불구하고 러·우 전쟁으로 화석연료 가격이 급등하면 서 재생에너지 상대가격은 개선되고 있다. 2022년 1분기 기준 태양광과 육상풍력의 발전비용은 신규 석탄 및 가스와 비교하여 1/2 수준이다([그림 2-10]). 2022년 상반기 기준 태양광, 육상풍력, 배터리저장장치의 LCOE는 각각 $45/MWh, $46/MWh, $153/MWh으로, 전년도 동기 대비 13.5%, 6.7%, 8.4% 증가하였다. 해상풍력 LCOE는 $86/MWh로 분석한 재생에너지원 중 유일하게 하 락세(-3.5%)를 보였다([그림 2-10]).14) 2020년 이후 태양광 모듈 비용은 와트당 26~27달러를 유지하였지만, 기타 BOS(Balance of System), 물류비, 인건비가 상 승하여 태양광 LCOE가 증가하였고, 육상풍력은 중국지역에서의 LCOE는 하락하였 으나, 이외 지역에서 철강, 원자재, 인건비, 물류비 등이 상승하면서 글로벌 육상풍 력 LCOE 상승을 이끌었다. 배터리 저장장치의 LCOE 상승은 리튬 중심의 원자재 비용과 충전 비용 상승이 주요 이유였다. 해상풍력 LCOE 하락세는 중국 중심의 보 급(약 85%)이 이루어졌기 때문으로, 중국에서의 보조금 지급이 풍력 부품 가격 하 락을 초래한 것으로 보인다. 다만 중국 이외 지역의 해상풍력 LCOE는 상승한 것으 로 나타났다.15) 14) BNEF(2022d) 엑셀자료를 참고하여 작성. 15) BNEF(2022e), pp.3~4. 14"
3,ebp-poc-all-index,0.004000,2021,"참고문헌 매일경제. EU, 중국산 타워 반덤핑 확정으로 수혜. https://m.mk.co.kr/stockview/?sCode=21&t_uid=5&c_uid=125287 (최종접속일: 2021.2.24). 매일경제. 웅진에너지 결국 상장폐지 결정. https://www.mk.co.kr/news/stock/view/2020/05/500645/ (최종접속일: 2021.11.18). 미국 에너지부. Hydrogen Program. https://www.hydrogen.energy.gov/ (최종접속일: 2021.10.11). 법률신문. 기후변화 대응을 위한 EU 탄소국경조정메커니즘(CBAM, 소위 탄소국경세) 도입. https://m.lawtimes.co.kr/Content/LawFirm-NewsLetter?serial=171944 (최종 검색일: 2021. 12.15). 연합뉴스. 인도 “개발도상국, 화석연료 쓸 자격”...COP26서 ‘석탄 중단’ 뒤집어. https://www.yna.co.kr/view/AKR20211114025700077?input=1179m (최종접속일: 2021.11.14). 오피넷. 평균판매가격. https://www.opinet.co.kr/user/dopospdrg/dopOsPdrgSelect.do (최종접속일: 2021.11.01). 외교통상부. 대한민국과 유럽연합 및 그 회원국 간의 자유무역협정. https://fta.go.kr/webmodule/_PSD_FTA/eu/doc/00_3_Fulltext.PDF (최종접속일: 2021.10.31). 인더스트리 뉴스. EU의 중국 태양광 제품 수입제한 조치 종료. https://www.industrynews.co.kr/news/articleView.html?idxno=26322#:~:text=E U%EB%8A%94%20%EC%A7%80%EB%82%9C%202012%EB%85%84,%EB%8B%AC% EB%9D%BC%EB%8A%94%20%EC%9A%94%EA%B5%AC%EB%A5%BC%20%EB%B0% 9B%EC%95%84%EB%93%A4%EC%97%AC (최종접속일: 2021.12.5). 이투뉴스. [탐방] 한국형 블레이드로 풍력 활성화 '정조준' https://www.e2news.com/news/articleView.html?idxno=236114 (최종접속일: 2021.11.22). 일본 환경성 홈페이지. CO 배출 감축 대책 강화 유도형 기술개발·실증사업. 2 https://www.env.go.jp/earth/ondanka/cpttv_funds/outline/ (최종접속일: 131"
4,ebp-poc-all-index,0.003900,2022,"제2장 국제 재생에너지 시장 동향 높고 건설 기간이 길어 이러한 LCOE 하락을 이끌었을 것으로 보인다. 둘째, 많은 프로젝트에서 예비비가 원자재 상승과 물류비용 상승을 일부 상쇄하였을 것이다. 셋 째, 기술 개발과 효율성 및 규모의 개선이 이루어졌다. 넷째, 재생에너지 시장을 주 도하는 중국 시장의 재생에너지 비용 하락이 이루어졌으며, 다만 제조업체의 이익은 줄어든 것으로 보인다.13) [그림 2-9] 유틸리티급 재생에너지원별 LCOE와 화석연료 가격과의 비교 자료: IRENA(2022), p.32 <표 2-1> 설치비용, 이용률, LCOE 재생에너지원별 변화 (2010년~2021년) 설치비용 이용율 LCOE (2021 USD/kW) (%) (2021 USD/kWh) 2010 2021 변화율 2010 2021 변화율 2010 2021 변화율 바이오 2,714 2,353 -13% 72 68 -6% 0.078 0.067 -14% 지열 2,714 3,991 47% 87 77 -11% 0.05 0.068 34% 수력 1,315 2,135 62% 44 45 2% 0

이전에 어휘분석으로 검색한 결과는 아래와 같습니다. 위의 표와 비교해보시길 바랍니다.

In [32]:
print("<<사용자 입력 쿼리 문장>>: ", query)
result_to_dataframe(response_lexical_only)

<<사용자 입력 쿼리 문장>>:  2020년 LCOE 값을 알려줘.


,index_name,score,type,text
0,ebp-poc-all-index,8.486300,2021,"2. 재생에너지원별 가격 동향 2.1. 원별 균등화발전비용(LCOE)42) 2020년은 코로나19로 인한 전 세계적인 영향을 받았으나 재생에너지 보급 비용 이 하락하는 추세에는 큰 영향을 주지 못하였다. 2020년 균등화발전비용(levelized cost of electricity, 이하 LCOE)를 보면 전년 대비 태양광은 7% 하락하였고, 해상 풍력과 육상풍력은 각각 9%, 13% 하락하였다. 집광형 태양열발전(Concentrated Solar Power, 이하 CSP)은 상대적으로 적은 보급이지만 가장 많은 16%의 가격하 락을 보였다([그림 2-27]). [그림 2-27] 유틸리티급 재생에너지 원별 LCOE 변화(2019~2020년) 주: CSP의 경우 2018~2020년 연평균 변화율임. 자료: IRENA(2021b), p.25. 재생에너지 비용은 태양광과 풍력의 경우 기술의 발전, 규모의 경제, 가치사슬의 경쟁, 개발사의 경험 축적 등으로 지난 10년간 급속히 하락하였다. 가장 비용이 많이 하락한 태양광의 경우 LCOE는 2010년 $0.381/kWh에서 2020년 $0.057/kWh로 85% 하락하였다. 육상풍력과 해상풍력 비용도 빠르게 하락하였는데 육상풍력의 경 우 동기간 $0.089/kWh에서 $0.039/kWh로 56%, 해상풍력은 $0.162 /kWh에서 $0.084/kWh로 48% 하락한 것으로 나타났다. CSP의 경우 보급이 많지 않지만 비 42) LCOE는 발전량 한 단위당 평균 발전비용으로 발전시설 운영 총비용의 현재 값을 총발전량의 현재 값으로 나누어 계산함. 26"
1,ebp-poc-all-index,6.560200,2021,"는 것이다. 동기간 육상풍력의 학습률은 32%, 해상풍력의 학습률은 15%에 이른 다.44) [그림 2-28]은 유틸리티급 재생에너지 원별 LCOE 추세를 연도별로 보여주는 것 이다. 가장 비용이 많이 하락한 태양광의 경우 추세가 안정적으로 매년 하락하였음 이 눈에 띈다. 그림에서 회색밴드는 화석연료 발전비용 범위를 나타내는 것으로 LCOE 범위가 회색밴드에 포함될 경우, 해당 재생에너지원의 발전비용이 화석연료 발전비용 범위 내에 속하게 되는 것으로 화석연료와 비교하여 경쟁력을 갖춘 것으 로 해석할 수 있다. 2020년 현재 모든 재생에너지원의 LCOE가 화석연료 발전비용 범위 내에 들어와서, 재생에너지원은 이미 화석연료 대비 가격 경쟁력을 갖춘 것으 로 볼 수 있다. 특히 육상풍력의 경우 화석연료 발전비용 최저보다 낮은 수준이다. [그림 2-28] 유릴리티급 재생에너지 원별 LCOE 변화(2010~2020년) 자료: IRENA(2021b), p.33. 44) 자료: IRENA(2021b), p.19. 28"
2,ebp-poc-all-index,5.815600,2022,"1.3. 재생에너지원별 균등화발전비용(LCOE)8) IRENA(2022)에 따르면 2021년에는 원자재와 물류비용 상승 등의 이슈가 있었 지만, 태양광, 육상풍력, 해상풍력 등의 비용 하락 추세는 유지되었다. 태양광은 부품 가격이 상승하였지만 균등화발전비용(Levelized Cost of Electricity, 이하 LCOE) 하락 추세에는 큰 영향을 미치지 못하였고, 육상풍력은 전년대비 이용율이 크게 개 선되면서 비용이 하락하였다.9) 2021년 LCOE를 보면 태양광은 전년대비 13% 감소 하였고, 해상풍력은 13%, 육상풍력은 15% 감소하였지만, CSP(Concentrating solar power)는 7% 증가하였다.10) CSP는 전세계적으로 3건의 설비가 가동 개시 되었는데, 이중 칠레의 CSP 설비가 건설 지연으로 비용이 증가하여 2010년~2015년 사이에 개발된 프로젝트와 비용이 비슷한 수준을 보였다.11) 재생에너지 비용은 지난 11년 동안 지열과 수력을 제외하고 모두 감소하면서, 재 생에너지원 모두 화석발전 비용과 비교하여 경쟁력을 갖춘 것으로 나타났다([그림 2-9]). LCOE가 가장 많이 하락한 에너지원은 태양광으로, 2010년 $0.417/kWh에 서 2021년 $0.048/kWh로 88% 하락하였다. 2021년 육상풍력 LCOE는 $0.033/kWh, 해상풍력은 $0.075/kWh로 지난 11년 동안 각각 68%와 60% 하락 한 반면, 지열과 수력의 LCOE는 각각 $0.068/kWh, $0.048/kWh로 지난 11년동 안 34%와 24% 상승하였다. 바이오는 지난 11년 동안 가격 분산이 감소하고 있으 나 상승이나 하락 등의 추세는 관측되지 않았다(<표 2-1>). 여기서 태양광과 육상풍 력의 가격하락은 기술개발의 이유가 가장 크고, 해상풍력은 최근 중국의 해상풍력 발전설비 설치 비중이 2021년 82%까지 증가함에 따라 중국의 상황이 대폭 반영된 특징이 있다.12) 2021년 원자재 가격은 상승하였지만 태양광과 육상풍력 LCOE가 하락할 수 있었 던 이유는 다음과 같다. 첫째, 원자재 가격 상승은 2020년 하반기에서 2021년 상 반기에 이루어졌으며, 2021년 가동개시된 설비는 원자재 가격 상승 이전에 건설되 었을 가능성이 높다. 특히 유럽 외 지역에서 건설된 대규모 프로젝트들은 구매력이 8) LCOE는 발전량 한 단위당 평균 발전비용으로 발전시설 운영 총비용의 현재 값을 총발전량의 현재 값으로 나누어 계산함(공지영· 조일현, 2021, p.26) 9) IRENA(2022), p.16. 10) 전게서, p.16. 11) 다만 해당 설비는 대규모 설비이므로 LCOE는 2건의 중국 CSP 설비에 비해 약간 높은 수준에 그쳤음(IRENA, 2022, p.33) 12) IRENA(2022), pp.30~31. 12"
3,ebp-poc-all-index,5.585500,2022,"<표 4-2> 변수 설명 변수명 설명 출처 전체 발전설비 용량 중 태양광, 풍력, 수력, 재생에너지 비중 Enerdata 조력, 지열, 바이오매스 발전 용량 비중 전체 발전설비 용량 중 태양광, 풍력 발전 변동성 재생에너지 비중 Enerdata 용량 비중 FiT/FiP 제도 시행시 1의 값을 갖는 FiT/FiP REN21, BNEF 더미변수 RPS RPS 제도 시행시 1의 값을 갖는 더미변수 REN21, BNEF 경매 경매제도 시행시 1의 값을 갖는 더미변수 REN21, BNEF 제조업 비중 GDP에서 제조업이 차지하는 비중 World Bank GDP per cap 1인당 GDP(2015년 US dollar) World Bank 금융발전도 GDP 대비 민간신용 비중 World Bank 석탄, 원유, 천연가스, 광물 및 임업제품의 천연자원 지대 비중 World Bank 지대가 GDP에서 차지하는 비중 BP Statistical Review of 전력 순수출량이 0보다 큰 경우 1의 값을 해외 계통연계 World Energy & 갖는 더미변수 Ember156) CO2 per cap 1인당 CO2 배출량 World Bank 국가 면적 국가 면적 World Bank 156) Our World in Data에서 BP Statistical Review of World Energy와 Ember 자료를 가공한 데이터를 활용함. 78"
4,ebp-poc-all-index,5.581000,2021,"제2장 국제 신재생에너지 보급 및 가격 동향 [그림 2-32] BNEF 추정 LCOE 변화(2010~2020년) 자료: BNEF(2021g), p.4. 2.2. 지역별 LCOE 오늘날 전 세계 인구의 2/3가 육상 풍력 또는 대규모 태양광(또는 둘 다)이 가장 저렴한 신규 대규모 전력원인 국가에 거주하고 있다. 하지만 여전히 일본, 한국, 인 도네시아, 말레이시아, 필리핀은 석탄이 가장 저렴한 신규 대규모 발전원이다([그림 2-33]). 기존 화석연료 전원의 한계 비용(기존 설비 운영)을 비교했을 때도 중국, 인 도, 독일, 프랑스, 이탈리아, 스페인, 포르투갈, 그리스의 경우 현재 기존 화석연료 발전소(석탄 및 가스)를 가동하는 것보다 신규 대규모 태양광을 건설하는 것이 더 저렴하고, 스웨덴 영국, 네덜란드

## 8. LangChain을 이용한 Question & Answer

- langchain의 similarity_search_with_score API를 활용하는 방법
    - [API: similarity_search_with_score](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html#langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.similarity_search)


In [33]:
from langchain.chains.question_answering import load_qa_chain

In [34]:
results = vector_db.similarity_search_with_score(
    query=query,
    k=5,
    search_type="approximate_search",
    boolean_filter={
        "bool": {
            "filter": []
        }
    }
)

In [35]:
[res[0].page_content for res in results[:3]]

['[그림 2-31] 유틸리티급 태양광 LCOE 비용 변화(2010~2020년) 자료: IRENA(2021b), p.83. 비용 하락이 가장 두드러지고 최근 가장 보급이 활발한 태양광의 지난 10년간의 LCOE 변화를 분해해 보면 태양광 모듈비용 하락이 46%를 차지하여 가장 중요한 요인이었다. 태양광 모듈의 가격 하락은 모듈 생산의 전 가치사슬에서 기술적 진보 로 인한 것이다. 그리고 인버터, 구조물, 나머지 BoS 비용이 18%를 차지하였고 개 발 EPC, 그 외 소프트 비용이 26%를 차지하였다. 나머지는 시장이 성숙해짐에 따 라 금융비용 하락, O&M 비용 하락, 태양광 이용률이 높은 곳의 개발이 증가함에 따라 높아진 이용률이 비용하락에 기여하였다.45) 주요 기관마다 LCOE 추정은 조금씩 다르다. BNEF의 태양광과 풍력의 2021년 상반기 기준 추정을 보면,46) 고정식 태양광은 $0.48/kWh, 추적식 태양광 $0.38/kWh, 육상풍력 $0.41/kWh, 해상풍력 $0.82/kWh이다([그림 2-32]). 이는 <표 2-2>의 IRENA 추정보다는 낮다. 여기서 눈여겨볼 것은 전통적으로 추정되던 태양광과 풍 력 등 재생에너지원에 대한 LCOE와 더불어, 태양광과 풍력이 변동성 재생에너지임 에 따라 ESS의 보급도 중요해지면서 ESS의 LCOE도 추정하여 제시한다는 것이다. 2021년 상반기 ESS의 LCOE는 $138/kWh로 최근 비용이 빠르게 하락하는 것을 볼 수 있다([그림 2-32]).47) 45) IRENA(2021b), p.83. 46) BNEF는 반기로 LCOE를 업데이트함. 47) ESS는 유틸리티급 리튬이온 배터리로 충전비용을 포함하여 일일 4시간 활용 가정. 30',
 '[그림 2-33] 2021년 LCOE 기준 최저비용 신규 대규모 발전원 자료: BNEF(2021g), p.15. [그림 2-34] 2021년 LCOE 기준 신규 태양광 풍력 VS 기존 화석연료 자료: IBNEF(2021g), p.16. 32',
 'BNEF(

### 사용자 정의 가능한 옵션
이제 벡터 저장소가 준비되었으므로 질문을 시작할 수 있습니다.

Vector Store를 둘러싸서 LLM 입력을 받는 LangChain에서 제공하는 래퍼를 사용할 수 있습니다.
이 래퍼는 뒤에서 다음 단계를 수행합니다.
- 질문을 입력합니다.
- 질문 임베딩 생성
- 관련 문서 가져오기
- 프롬프트에 문서와 질문을 채워 넣습니다.
- 프롬프트로 모델을 호출하고 사람이 읽을 수 있는 방식으로 답변을 생성합니다.

위 시나리오에서는 질문에 대한 상황 인식 답변을 빠르고 쉽게 얻을 수 있는 방법을 탐색했습니다. 이제 문서를 가져오는 방법을 사용자 정의할 수 있는 [RetrievalQA](https://python.langchain.com/en/latest/modules/chains/index_examples/Vector_db_qa.html)의 도움으로 더 사용자 정의 가능한 옵션을 살펴보겠습니다. `chain_type` 매개변수를 사용하여 프롬프트에 추가해야 합니다. 또한 검색해야 하는 관련 문서 수를 제어하려면 아래 셀에서 'k' 매개변수를 변경하여 다른 출력을 확인하세요. 많은 시나리오에서 LLM이 답변을 생성하는 데 사용한 소스 문서가 무엇인지 알고 싶을 수 있습니다. LLM 프롬프트의 컨텍스트에 추가된 문서를 반환하는 `return_source_documents`를 사용하여 출력에서 ​​해당 문서를 가져올 수 있습니다. 'RetrievalQA'를 사용하면 모델에 특정한 사용자 정의 [프롬프트 템플릿](https://python.langchain.com/en/latest/modules/prompts/prompt_templates/getting_started.html)을 제공할 수도 있습니다.

참고: 이 예에서는 Amazon Bedrock에서 LLM으로 Anthropic Claude를 사용하고 있습니다. 이 특정 모델은 입력이 'Human:' 아래에 제공되고 모델이 'Assistant:' 다음에 출력을 생성하도록 요청되는 경우 가장 잘 수행됩니다. 아래 셀에는 LLM이 기본 상태를 유지하고 컨텍스트 외부에서 응답하지 않도록 프롬프트를 제어하는 ​​방법의 예가 나와 있습니다.

#### [[REF] Using langchain for Question Answering on Own Data](https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed)

In [36]:
from langchain.schema import BaseRetriever
from typing import Any, Dict, List, Optional, List, Tuple
from langchain.callbacks.manager import CallbackManagerForRetrieverRun

# lexical(keyword) search based (using Amazon OpenSearch)
class OpenSearchLexicalSearchRetriever(BaseRetriever):
    os_client: Any
    index_name: str
    k = 3
    filter = []

    def normalize_search_results(self, search_results):
        hits = (search_results["hits"]["hits"])
        max_score = float(search_results["hits"]["max_score"])
        for hit in hits:
            hit["_score"] = float(hit["_score"]) / max_score
        search_results["hits"]["max_score"] = hits[0]["_score"]
        search_results["hits"]["hits"] = hits
        return search_results

    def update_search_params(self, **kwargs):
        self.k = kwargs.get("k", 3)
        self.filter = kwargs.get("filter", [])
        self.index_name = kwargs.get("index_name", self.index_name)

    def _reset_search_params(self, ):
        self.k = 3
        self.filter = []
        
    def query_lexical(self, query, filter=[], k=5):
        QUERY_TEMPLATE = {
            "size": k,
            "query": {
                "bool": {
                    "must": [
                        {
                            "match": {
                                "text": {
                                    "query": query,
                                    "operator":  "or"
                                }
                            }
                        }
                    ],
                    "filter": filter
                }
            }
        }
        
        if len(filter) > 0:
            QUERY_TEMPLATE["query"]["bool"]["filter"].extend(filter)
            
        return QUERY_TEMPLATE
    

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun) -> List[Document]:
        
        query = self.query_lexical(
            query=query,
            filter=self.filter,
            k=self.k
        )

        # print ("lexical search query: ")
        # print(query)
        
        search_results = self.os_client.search(
            body=query,
            index=self.index_name
        )

        results = []
        if search_results["hits"]["hits"]:
            search_results = self.normalize_search_results(search_results)
            for res in search_results["hits"]["hits"]:

                metadata = res["_source"]["metadata"]
                metadata["id"] = res["_id"]

                doc = Document(
                    page_content=res["_source"]["text"],
                    metadata=metadata
                )
                results.append((doc))

        self._reset_search_params()

        return results[:self.k]


In [37]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
# from utils.rag import run_RetrievalQA, show_context_used

In [38]:
prompt_template = """
\n\nHuman: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}

\n\nAssistant:"""


PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [39]:
chain = load_qa_chain(
    llm=llm_text,
    chain_type="stuff",
    prompt=PROMPT,
    verbose=True
)

In [40]:
boolean_filter = []
boolean_filter = [
    {"term": {"metadata.source": "국제 신재생에너지 정책변화 및 시장 분석_22-26.pdf"}},
    {"term": {"metadata.type": "국제 신재생에너지 정책변화 및 시장 분석"}},
]

In [41]:
opensearch_lexical_retriever = OpenSearchLexicalSearchRetriever(
    os_client=os_client,
    index_name=index_name,
    k=3,
    filter=boolean_filter
)

In [42]:
answer = chain.invoke(
    {
        "input_documents": opensearch_lexical_retriever.get_relevant_documents(query), 
        "question": query
    }, 
    # return_only_outputs=True
)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.



Question: 2020년 LCOE 값을 알려줘.



Assistant:
죄송합니다. 제가 제공받은 정보로는 2020년 LCOE(Levelized Cost of Energy, 균등화발전원가) 값을 알려드릴 수 없습니다. LCOE는 특정 발전원의 전력 생산 비용을 나타내는 지표이지만, 해당 연도의 구체적인 값이 제시되지 않았기 때문입니다. 더 자세한 정보가 필요할 것 같습니다.
> Finished chain.

> Finished chain.


In [43]:
opensearch_semantic_retriever = vector_db.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 5,
        "boolean_filter": boolean_filter
    }
)

In [44]:
answer = chain.invoke(
    {
        "input_documents": opensearch_semantic_retriever.get_relevant_documents(query), 
        "question": query
    }, 
    # return_only_outputs=True
)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.



Question: 2020년 LCOE 값을 알려줘.



Assistant:
죄송합니다. 제가 제공받은 정보로는 2020년 LCOE(Levelized Cost of Energy, 균등화발전원가) 값을 알려드릴 수 없습니다. LCOE는 특정 발전원의 전력 생산 비용을 나타내는 지표이지만, 해당 연도의 구체적인 값이 제시되지 않았기 때문입니다. 더 자세한 정보가 필요할 것 같습니다.
> Finished chain.

> Finished chain.


## 9. OpenSearch Hybrid 검색

OpenSearch Hybrid 는 아래와 같은 방식으로 작동합니다.
- (1) "Vector 서치" 하여 스코어를 얻은 후에 표준화를 하여 스코어를 구함. 
    - 전체 결과에서 가장 높은 스코어는 표준화 과정을 통하여 스코어가 1.0 이 됨.
- (2) Keyword 서치도 동일하게 함.
- (3) Reciprocal Rank Fusion (RRF) 기반 Re-rank
    - Paper: https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf
    - Desc: https://medium.com/@sowmiyajaganathan/hybrid-search-with-re-ranking-ff120c8a426d
    - **RRF의 경우 score가 아닌 ranking 정보를 활용, 때문에 score normalization이 필요 없음**

RRF는 langchain에서 "Ensemble Retriever" 이름으로 api를 제공합니다. 
- https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble


### Ensemble retriever 정의
- https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble
- RRF 방식만 지원
- Rank constant (param "c")
    - This value determines how much influence documents in individual result sets per query have over the final ranked result set. A higher value indicates that lower ranked documents have more influence. This value must be greater than or equal to 1. Defaults to 60.
    - 숫자 높을 수록 낮은 랭크의 문서가 더 중요시 된다

In [45]:
from langchain.retrievers import EnsembleRetriever

In [46]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[opensearch_lexical_retriever, opensearch_semantic_retriever],
    weights=[0.5, 0.5],
    c=100,
    k=5
)

In [48]:
# 쿼리 지정
query = """
You are a data analyst and you are conducting a PESTEL analysis of the solar power industry.

Please follow the steps bellow.

1. Gather and analyze data related to the political, economic, social, technological, environmental, and legal factors influencing the Solar Photovoltaics industry. Specifically focus on:


**Political Factors:**
- Analyze government policies and regulations that impact the Solar Photovoltaics sector
- Evaluate political stability and its effect on Solar Photovoltaics investments
- Examine international agreements and treaties that influence Solar Photovoltaics development

**Economic Factors:**
- Study the cost competitiveness of Solar Photovoltaics technologies compared to traditional sources
- Analyze trends in financing and investment in Solar Photovoltaics projects
- Assess economic growth rates and their impact on Solar Photovoltaics demand

**Social Factors:**
- Investigate public perception and acceptance of Solar Photovoltaics solutions
- Examine demographic trends and their influence on Solar Photovoltaics adoption
- Evaluate social equity considerations in Solar Photovoltaics access and distribution

**Technological Factors:**
- Assess advancements in Solar Photovoltaics technologies and their potential for growth
- Analyze research and development activities in the Solar Photovoltaics sector
- Evaluate technological barriers and opportunities for innovation in Solar Photovoltaics

**Environmental Factors:**
- Study the environmental impact of Solar Photovoltaics sources compared to fossil fuels
- Analyze the role of Solar Photovoltaics in addressing climate change and sustainability
- Evaluate environmental regulations and policies affecting the Solar Photovoltaics industry

**Legal Factors:**
- Examine laws and regulations governing Solar Photovoltaics development and operations
- Assess intellectual property rights and legal challenges in the Solar Photovoltaics sector
- Analyze international trade agreements and their impact on Solar Photovoltaics markets

**Data Analysis Tasks:**
- Collect data from reliable sources such as government reports, industry publications, and research studies
- Conduct quantitative and qualitative analysis to identify key trends and patterns
- Create visualizations to present data insights and findings
- Compare data across different regions or countries to understand global trends
Provide recommendations based on the data analysis for strategic decision-making in the Solar Photovoltaics sector

2. Please translate in Korean.

"""

In [49]:
%%time
answer = chain.invoke(
    {
        "input_documents": ensemble_retriever.get_relevant_documents(query), 
        "question": query
    }
)

print("##############################")
print("query: \n", query)
print("answer: \n", answer)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Lessons from Europe and the US.” Energy Policy 35(1): 144-155. García-Álvarez, M. T., Cabeza-García, L., Soares, I. 2017. “Analysis of the promotion of onshore wind energy in the EU: Feed-in tariff or renewable portfolio standard?” Renewable energy 111: 256-264. Grinlinton, D., Paddock, L. 2010. “The role of feed-in tariffs in supporting the expansion of solar energy production.” The University of Toledo Law Review 41(4): 943-974. IRENA. 2022. “Renewable Power Generation Costs in 2021.” Menanteau, P., Finon, D., Lamy, M. L. 2003. “Prices versus quantities: choosing policies for promoting the development of renewable energy.” Energy policy 31(8): 799-812. Neij, L. 1997. “Use of experience cu